In [1]:
import os
import glob
import pandas as pd
from sqlalchemy import create_engine

# CSV Data

In [2]:
path = 'C:\\Users\\MAQ\\New-Business\\RCS\\Sample\\'
name = 'RCSLNK-MQ07.csv'

### Compare Columns

In [3]:
SQLCols = ["Acctnumber", "SSN", 
           "NAME", "Address", "Address2", "City", "State", "Zip", 
           "DateOfBirth", "HomePhone", "WorkPhone", "OtherPhone", "CellPhone", 
           "EffectiveDate", "CurrBalance", "LastPayDate", "LastPayAmt", "PlaceBatchID", 
           "ChgOffBalance", "ChgOffDate", "OwnerName", "OriginalCreditor", 
           "OutOfStatuteDate", "SendCBRNegNotice", "SendOOSNotice", 
           "ChargeOffCreditorName"]

### CSV Columns

In [4]:
CSVCols = ["AcctID", "Acctnumber", "SSN", 
           "FirstName", "LastName", "Address", "Address2", "City", "State", "Zip", 
           "DateOfBirth", "HomePhone", "WorkPhone", "OtherPhone", "CellPhone", 
           "EffectiveDate", "CurrBalance", "LastPayDate", "LastPayAmt", "PlaceBatchID", 
           "ChgOffBalance", "ChgOffDate", "OwnerName", "OriginalCreditor", 
           "OutOfStatuteDate", "SendCBRNegNotice", "SendOOSNotice", 
           "ChargeOffCreditorName"]

CSVTextCols = ["NAME", "Address", "Address2", "City", "State", "OwnerName", 
               "OriginalCreditor", "ChargeOffCreditorName"]

In [5]:
def dataCSV(path, name, cols):

    df = pd.read_csv(path + name, dtype=str)

    # Filters only required column

    df = df[cols]

    # Concatename First and Last name column into a new column NAME

    df['NAME'] = df.apply(lambda row: f"{row['LastName']},{row['FirstName']}", axis=1)
    
    # Sets AcctID as index
    
    df.set_index('AcctID', inplace=True)
    
    # Rearranges column
    
    df = df[SQLCols]
    
    # sorts dataframe according to index
    
    df.sort_index(inplace=True)

    # Changes all text to UpperCase

    df[CSVTextCols] = df[CSVTextCols].apply(lambda x: x.str.upper())

    # Replace NaN values with empty strings
    
    df.fillna('', inplace=True)
    
    return df

In [6]:
dfCSV = dataCSV(path, name, CSVCols)

# Accounts

In [7]:
def Accounts(path, name):

    df = pd.read_csv(path + name, dtype=str)
    
    # Converts column of AcctID to a list
    # with inverted commas around AcctID's

    AcctIDList = (df['AcctID'].apply(lambda id: f"'{id}'")).tolist()

    # Converts the list to a string for SQL query

    AcctIDs = ','.join(AcctIDList)

    return AcctIDs

# SQL Data

In [8]:
date = '11-29-2023'
accString = Accounts(path, name)

In [9]:
def dataSQL(date, accString):

    # Configuration for SQL engine

    server = 'unifin-sql'
    usr = 'read_only'
    pwd = 'Neustar01'
    db = 'tiger'
    driver = 'ODBC+Driver+17+for+SQL+Server'

    config = f"mssql+pyodbc://{usr}:{pwd}@{server}/{db}?driver={driver}"

    engine = create_engine(config)

    # -----------------------DataFrame One----------------------- #

    # Query for fetching Account Number

    query1 = "SELECT DBR_CLI_REF_NO AS 'AcctID', \
              RIGHT(ADR_NAME, LEN(ADR_NAME) - 1) AS 'Acctnumber' \
              FROM CDS.DBR INNER JOIN CDS.ADR ON DBR_NO = ADR_DBR_NO \
              WHERE DBR_CLI_REF_NO IN (" + accString + ") AND ADR_SEQ_NO = 'R2' \
              AND DBR_ASSIGN_DATE_O = " + f"'{date}'"

    dfOne = pd.read_sql(query1, engine, dtype=str)

    dfOne.set_index('AcctID', inplace=True)

    # -----------------------DataFrame Two----------------------- #

    # Query for fetching Name, SSN, Address, DOB

    query2 = "SELECT DBR_CLI_REF_NO AS 'AcctID', \
              DBR_NAME1 AS 'NAME', \
              ADR_TAX_ID AS 'SSN', \
              ADR_ADDR1 AS 'Address', \
              ADR_ADDR2 AS 'Address2', \
              ADR_CITY AS 'City', \
              ADR_STATE AS 'State', \
              ADR_ZIP_CODE AS 'Zip', \
              CONVERT(VARCHAR,ADR_DOB_O,101) AS 'DateOfBirth' \
              FROM CDS.DBR INNER JOIN CDS.ADR ON DBR_NO = ADR_DBR_NO \
              WHERE DBR_CLI_REF_NO IN (" + accString + ") AND ADR_SEQ_NO = '01' \
              AND DBR_ASSIGN_DATE_O = " + f"'{date}'"
    
    dfTwo = pd.read_sql(query2, engine, dtype=str)

    dfTwo.set_index('AcctID', inplace=True)

    # ----------------------DataFrame Three---------------------- #

    # Query for Home Phone & Cell Phone

    query3 = "SELECT DBR_CLI_REF_NO AS 'AcctID', \
              ADR_PHONE1 AS 'HomePhone', \
              ADR_PHONE2 AS 'CellPhone' \
              FROM CDS.DBR INNER JOIN CDS.ADR ON DBR_NO = ADR_DBR_NO \
              WHERE DBR_CLI_REF_NO IN (" + accString + ") AND ADR_SEQ_NO = '01' \
              AND DBR_ASSIGN_DATE_O = " + f"'{date}'"

    dfThree = pd.read_sql(query3, engine, dtype=str)

    dfThree.set_index('AcctID', inplace=True)

    # -----------------------DataFrame Four---------------------- #

    # Query for Work Phone & Other Phone

    query4 = "SELECT DBR_CLI_REF_NO AS 'AcctID', \
              ADR_PHONE1 AS 'WorkPhone', \
              ADR_PHONE2 AS 'OtherPhone' \
              FROM CDS.DBR INNER JOIN CDS.ADR ON DBR_NO = ADR_DBR_NO \
              WHERE DBR_CLI_REF_NO IN (" + accString + ") AND ADR_SEQ_NO = '11' \
              AND DBR_ASSIGN_DATE_O = " + f"'{date}'"

    dfFour = pd.read_sql(query4, engine, dtype=str)

    dfFour.set_index('AcctID', inplace=True)
    
    # -----------------------DataFrame Five---------------------- #

    # Query for Other DBR columns

    query5 = "SELECT DBR_CLI_REF_NO AS 'AcctID', \
              CONVERT(VARCHAR,DBR_ASSIGN_DATE_O,101) AS 'EffectiveDate', \
              DBR_ASSIGN_AMT AS 'CurrBalance', \
              CONVERT(VARCHAR,DBR_CL_DATES_3_O,101) AS 'LastPayDate', \
              DBR_CL_CODES_3 AS 'LastPayAmt', \
              DBR_CL_MISC_3 AS 'PlaceBatchID', \
              DBR_CL_CODES_1 AS 'ChgOffBalance', \
              CONVERT(VARCHAR,DBR_LAST_CHG_DATE_O,101) AS 'ChgOffDate', \
              DBR_CL_MISC_1 AS 'OwnerName', \
              DBR_CL_MISC_2 AS 'OriginalCreditor', \
              CONVERT(VARCHAR,DBR_CL_DATES_2_O,101) AS 'OutOfStatuteDate', \
              DBR_NegNotice_FLAG AS 'SendCBRNegNotice', \
              DBR_2_FLAG AS 'SendOOSNotice', \
              DBR_CL_MISC_2 AS 'ChargeOffCreditorName' \
              FROM CDS.DBR WHERE DBR_CLI_REF_NO IN (" + accString + ") \
              AND DBR_ASSIGN_DATE_O = " + f"'{date}'"

    dfFive = pd.read_sql(query5, engine, dtype=str)

    dfFive.set_index('AcctID', inplace=True)

    # ---------------------Merging DataFrames-------------------- #

    merged_df = pd.concat([dfOne, dfTwo, dfThree, dfFour, dfFive], axis=1)
    
    # sorts dataframe's columns
    
    sorted_df = merged_df[SQLCols]
    
    # sorts dataframe according to index
    
    sorted_df.sort_index(inplace=True)

    # Replaces all whitespace string with empty

    pattern = '^\s*$'

    replaced_df = sorted_df.replace(to_replace =pattern, value = '', regex = True)

    # Replace NaN values with empty strings
    
    df = replaced_df.fillna('')

    return df

In [10]:
dfSQL = dataSQL(date, accString)

In [11]:
def Comparison(dfCSV, dfSQL):

    # Comparison dataframe

    dfCompare = dfCSV == dfSQL

    dfCompare = dfCompare.add_suffix('_Match')

    # Renames both Dataframes

    df1 = dfCSV.add_suffix('_CSV')

    df2 = dfSQL.add_suffix('_SQL')

    # Columns for comparison

    cols = []

    for col in SQLCols: cols = cols + [col + '_CSV', col + '_SQL', col + '_Match']

    comparison_df = pd.concat([df1, df2, dfCompare], axis=1)
    
    # Rearranges columns

    compare_df = comparison_df[cols]
    
    compare_df.reset_index(inplace=True)
    
    return compare_df

In [12]:
compare = Comparison(dfCSV, dfSQL)

In [13]:
compare.to_csv(path + 'compare.csv', index=False)